# Load Libraries

In [1]:
import os
import warnings
import logging
import sys
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import dotenv
import pyet
import matplotlib.pyplot as plt

dotenv.load_dotenv()
logging.basicConfig(level=logging.INFO)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_colwidth', None)

# Load Data

In [2]:
DATA_FOLDER_PATH = os.getenv('DATA_FOLDER_PATH')
DAILY_DATA_NAME = os.getenv('DAILY_DATA_NAME')

if not DATA_FOLDER_PATH or not DAILY_DATA_NAME:
    logging.error("Environment variables DATA_FOLDER_PATH or DAILY_DATA_NAME are not set.")
    sys.exit(1)

DATA_PATH = os.path.join(DATA_FOLDER_PATH, DAILY_DATA_NAME + '.parquet')

try:
    data = pd.read_parquet(path=DATA_PATH)
    logging.info(f"Data loaded successfully from {DATA_PATH}")
except Exception as e:
    logging.error(f"Error loading data from {DATA_PATH}: {e}")
    sys.exit(1)

INFO:root:Data loaded successfully from D:\\Pooya\\Storegs\\Dropbox\\IRIMO\\Export\Iran_Daily_Data_1951_2025.parquet


# Data Info

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21097489 entries, 0 to 21097488
Data columns (total 26 columns):
 #   Column             Dtype         
---  ------             -----         
 0   date               datetime64[ns]
 1   station_id         object        
 2   station_name       object        
 3   region_id          object        
 4   region_name        object        
 5   lat                float64       
 6   lon                float64       
 7   station_elevation  float64       
 8   tmax               float64       
 9   tmin               float64       
 10  tm                 float64       
 11  umax               float64       
 12  umin               float64       
 13  um                 float64       
 14  ffm                float64       
 15  sshn               float64       
 16  rrr24              float64       
 17  pm                 float64       
 18  p0m                float64       
 19  ewm                float64       
 20  radglo24           flo

In [4]:
print("Number of unique region:", data['region_name'].nunique())
print("Number of unique station:", data[['region_name', 'station_name', 'station_id']].drop_duplicates().shape[0])

Number of unique region: 32
Number of unique station: 773


# Calculate ETo

### Required Functions

In [6]:
def data_quality_control(
    df, 
    required_vars = ['tm']
):
    df.dropna(subset=required_vars, inplace=True, how='any')
    return df

In [7]:
def calculate_eto(group, name, func, required_vars):
    station = group['station_name'].iloc[0]
    print(f"Calculating ETo For Station {station} ({name})")
    
    group = data_quality_control(
        df=group, 
        required_vars=required_vars
    )
    group = group.set_index('date').sort_index()
    
    if group.empty:
        return pd.Series(np.nan, index=group.index)
    
    missing = [var for var in required_vars if var not in group.columns]
    if missing:
        raise ValueError(f"Required variable(s) missing from the data: {missing}")

    kwargs = {}
    if "lat" in required_vars:
        kwargs["lat"] = pyet.utils.deg_to_rad(lat=group['lat'].iloc[0])
    if "station_elevation" in required_vars:
        kwargs["elevation"] = group['station_elevation'].iloc[0]
    if "sshn" in required_vars:
        kwargs["rs"] = pyet.calc_rad_sol_in(n=group["sshn"], lat=pyet.utils.deg_to_rad(lat=group['lat'].iloc[0]))
    if "ffm" in required_vars:
        kwargs["wind"] = group["ffm"]
    if "tm" in required_vars:
        kwargs["tmean"] = group['tm']
    if "tmax" in required_vars:
        kwargs["tmax"] = group['tmax']
    if "tmin" in required_vars:
        kwargs["tmin"] = group['tmin']
    if "um" in required_vars:
        kwargs["rh"] = group['um']
    if "umin" in required_vars:
        kwargs["rhmin"] = group['umin']
    if "umax" in required_vars:
        kwargs["rhmax"] = group['umax']

    eto = func(**kwargs)
    
    return eto

In [8]:
def calculate_eto_all_methods(group, dic_eto):
    results = {}
    for method, meta in dic_eto.items():
        df = group.copy()
        res = calculate_eto(
            df,
            name=method,
            func=meta["func"],
            required_vars=meta["required_vars"]
        )
        results[method] = res
    return pd.DataFrame(results)

### ETo

In [9]:
dic_eto = {
    "Penman": {
        "func": pyet.penman,
        "required_vars": ["tm", "ffm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "Penman-Monteith": {
        "func": pyet.pm,
        "required_vars": ["tm", "ffm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "ASCE-PM": {
        "func": pyet.pm_asce,
        "required_vars": ["tm", "ffm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "FAO-56": {
        "func": pyet.pm_fao56,
        "required_vars": ["tm", "ffm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "Priestley-Taylor": {
        "func": pyet.priestley_taylor,
        "required_vars": ["tm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "Kimberly-Penman": {
        "func": pyet.kimberly_penman,
        "required_vars": ["tm", "ffm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "Thom-Oliver": {
        "func": pyet.thom_oliver,
        "required_vars": ["tm", "ffm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "Blaney-Criddle": {
        "func": pyet.blaney_criddle,
        "required_vars": ["tm", "lat"]
    },
    "Hamon": {
        "func": pyet.hamon,
        "required_vars": ["tm", "lat"]
    },
    "Romanenko": {
        "func": pyet.romanenko,
        "required_vars": ["tm", "tmax", "tmin", "um", "umin", "umax"]
    },
    "Linacre": {
        "func": pyet.linacre,
        "required_vars": ["tm", "lat", "station_elevation", "tmax", "tmin"]
    },
    # "Haude": {
    #     "func": pyet.haude,
    #     "required_vars": ["tmax", "um"]
    # },
    "Turc": {
        "func": pyet.turc,
        "required_vars": ["tm", "sshn", "um"]
    },
    "Jensen-Haise": {
        "func": pyet.jensen_haise,
        "required_vars": ["tm", "sshn"]
    },
    "Mcguinness-Bordne": {
        "func": pyet.mcguinness_bordne,
        "required_vars": ["tm", "lat"]
    },
    "Hargreaves": {
        "func": pyet.hargreaves,
        "required_vars": ["tm", "tmax", "tmin", "lat"]
    },
    "FAO-24": {
        "func": pyet.fao_24,
        "required_vars": ["tm", "ffm", "sshn", "um", "station_elevation"]
    },
    "Abtew": {
        "func": pyet.abtew,
        "required_vars": ["tm", "sshn"]
    },
    "Makkink": {
        "func": pyet.makkink,
        "required_vars": ["tm", "sshn", "station_elevation"]
    },
    "Oudin": {
        "func": pyet.oudin,
        "required_vars": ["tm", "lat"]
    }
}


eto = data.groupby(
    ['region_id', 'region_name', 'station_id', 'station_name'],
    group_keys=True
).apply(calculate_eto_all_methods, dic_eto)

Calculating ETo For Station Zidashte Taleghan (Penman)
Calculating ETo For Station Zidashte Taleghan (Penman-Monteith)
Calculating ETo For Station Zidashte Taleghan (ASCE-PM)
Calculating ETo For Station Zidashte Taleghan (FAO-56)
Calculating ETo For Station Zidashte Taleghan (Priestley-Taylor)
Calculating ETo For Station Zidashte Taleghan (Kimberly-Penman)
Calculating ETo For Station Zidashte Taleghan (Thom-Oliver)
Calculating ETo For Station Zidashte Taleghan (Blaney-Criddle)
Calculating ETo For Station Zidashte Taleghan (Hamon)
Calculating ETo For Station Zidashte Taleghan (Romanenko)
Calculating ETo For Station Zidashte Taleghan (Linacre)
Calculating ETo For Station Zidashte Taleghan (Turc)
Calculating ETo For Station Zidashte Taleghan (Jensen-Haise)
Calculating ETo For Station Zidashte Taleghan (Mcguinness-Bordne)
Calculating ETo For Station Zidashte Taleghan (Hargreaves)
Calculating ETo For Station Zidashte Taleghan (FAO-24)
Calculating ETo For Station Zidashte Taleghan (Abtew)
Ca

In [10]:
if isinstance(eto.index, pd.MultiIndex):
    eto = eto.reset_index()
    data = pd.merge(data, eto, on=['region_id', 'region_name', 'station_id', 'station_name', 'date'], how='left')
else:
    data = data.join(eto)

data

,date,station_id,station_name,region_id,region_name,lat,lon,station_elevation,tmax,tmin,tm,umax,umin,um,ffm,sshn,rrr24,pm,p0m,ewm,radglo24,evt,td_m,twet_m,tsoil_m,ewsm,Penman,Penman-Monteith,ASCE-PM,FAO-56,Priestley-Taylor,Kimberly-Penman,Thom-Oliver,Blaney-Criddle,Hamon,Romanenko,Linacre,Turc,Jensen-Haise,Mcguinness-Bordne,Hargreaves,FAO-24,Abtew,Makkink,Oudin
0,1951-01-01 12:00:00,18554,Zidashte Taleghan,ALKK,Alborz,36.13,50.68,2255.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1951-01-02 12:00:00,18554,Zidashte Taleghan,ALKK,Alborz,36.13,50.68,2255.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1951-01-03 12:00:00,18554,Zidashte Taleghan,ALKK,Alborz,36.13,50.68,2255.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1951-01-04 12:00:00,18554,Zidashte Taleghan,ALKK,Alborz,36.13,50.68,2255.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1951-01-05 12:00:00,18554,Zidashte Taleghan,ALKK,Alborz,36.13,50.68,2255.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21097484,2025-09-17 12:00:00,99440,Kahak,QOQM,Qom,34.40,50.87,1403.10,31.40,16.90,24.10,41.00,18.00,26.17,3.17,10.70,0.00,1005.84,858.92,7.92,2348.00,10.40,3.42,13.44,16.00,31.74,4.63,6.93,6.90,6.90,3.83,4.22,5.80,3.52,4.60,13.05,6.41,6.13,6.04,5.52,4.73,5.61,4.73,4.40,3.75
21097485,2025-09-18 12:00:00,99440,Kahak,QOQM,Qom,34.40,50.87,1403.10,30.90,16.70,23.80,33.00,18.00,24.25,3.25,10.80,0.00,1005.85,858.89,7.56,2351.00,9.90,2.72,13.30,15.00,31.94,4.60,7.01,6.98,6.98,3.71,4.19,5.80,3.47,4.49,13.29,6.28,6.23,5.98,5.42,4.62,5.64,4.73,4.39,3.69
21097486,2025-09-19 12:00:00,99440,Kahak,QOQM,Qom,34.40,50.87,1403.10,35.70,17.50,26.60,28.00,8.00,17.70,3.13,10.00,0.00,1000.59,855.40,6.28,2247.00,10.40,0.25,13.68,16.00,38.07,4.64,7.86,7.80,7.80,3.34,4.15,6.47,3.79,5.31,16.66,7.68,6.61,6.28,5.92,5.56,5.62,4.50,4.31,4.03
21097487,2025-09-20 12:00:00,99440,Kahak,QOQM,Qom,34.40,50.87,1403.10,32.60,19.40,26.00,27.00,5.00,18.00,3.14,11.00,0.00,1000.94,855.31,5.15,2341.00,10.60,-3.18,11.51,16.00,31.17,4.51,7.53,7.48,7.48,3.31,4.06,6.21,3.71,5.09,16.50,7.06,6.87,6.50,5.77,4.63,5.91,4.75,4.53,3.92


# Export Data

In [11]:
data.to_parquet(DATA_FOLDER_PATH + '/Iran_Daily_ETo_1951_2025.parquet')
logging.info("ETo data saved to pickle and parquet files.")

INFO:root:ETo data saved to pickle and parquet files.
